In [72]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [73]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [124]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [75]:
!ls

data		    yellow_tripdata_2023-01.parquet
introduction.ipynb  yellow_tripdata_2023-02.parquet


In [113]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


# Question 1 : Number of cloumns

In [114]:
len(df_jan.columns)

19

# Question2 : Computing duration

In [115]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']

df_jan['duration'].head()

0   0 days 00:08:26
1   0 days 00:06:19
2   0 days 00:12:45
3   0 days 00:09:37
4   0 days 00:10:50
Name: duration, dtype: timedelta64[ns]

In [116]:
df_jan['duration'] = df_jan['duration'].dt.total_seconds() / 60
df_jan['duration'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [117]:
# standard deviation of the trips duration in January
df_jan.duration.std()

42.59435124195458

# Question3:
Next, we need to check the distribution of the duration variable. 
There are some outliers. Let's remove them and keep only the records where the duration was
between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [126]:
df_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()

In [127]:
filter_size, columns = df_filtered.shape

In [128]:
raw_size, columns = df_jan.shape

In [129]:
round(((filter_size/raw_size) * 100), 0)

98.0

In [105]:
df_filtered.shape

(3009173, 20)

## Q4. One-hot encoding
### What's the dimensionality of this matrix (number of columns)?

In [130]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_filtered[categorical] = df_filtered[categorical].astype(str)

In [131]:
train_dicts = df_filtered[categorical].to_dict(orient='records')

In [132]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [138]:
print("number of columns:",  X_train.shape[1])

number of columns: 515


### Q5:  Training the model

#### Root mean square value is 

In [139]:
target = 'duration'
y_train = df_filtered[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261934850555

### Q6: evaluating the model

In [141]:
df_eval = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


In [142]:
df_eval['duration'] = df_eval.tpep_dropoff_datetime - df_eval.tpep_pickup_datetime
df_eval.duration = df_eval.duration.dt.total_seconds() / 60

In [143]:
df_eval['duration'].head()

0     1.683333
1     0.233333
2     0.233333
3    32.083333
4    13.300000
Name: duration, dtype: float64

In [146]:
df_eval.duration.std()

42.84210176105113

In [149]:
df_eval[categorical] = df_eval[categorical].astype('str')
eval_train = df_eval[categorical].to_dict(orient='records')
eval_train_dicts = dv.transform(eval_train)

In [150]:
y_values_eval = df_eval['duration']

y_pred = lr.predict(eval_train_dicts)

In [152]:
root_mean_squared_error(y_values_eval, y_pred)

42.284775704546334

In [167]:
# filtering the records to get the correct value
df_eval = df_eval[(df_eval.duration >= 1) & (df_eval.duration <= 60)].copy()

In [161]:
eval_train = df_eval[categorical].to_dict(orient='records')

In [162]:
eval_train_dicts = dv.transform(eval_train)

In [163]:
y_values_eval = df_eval['duration']


In [164]:
y_pred = lr.predict(eval_train_dicts)

#### Root mean squared value is 

In [165]:
root_mean_squared_error(y_values_eval, y_pred)

7.811817745843695